In [18]:
import formulas
import utilities
import pandas as pd
import os
import re


if __name__ == '__main__':

    website_creator_dir = os.getcwd()
    docs_dir = utilities.get_docs_path(website_creator_dir)
    formula_file_path = website_creator_dir + os.path.sep + 'formulas.csv'
    order_file_path = website_creator_dir + os.path.sep + 'sort_orders.csv'

    formulas_input_df = pd.read_csv(formula_file_path)
    sort_orders_df = pd.read_csv(order_file_path)
    formulas_df = formulas.get_formulas_df(formulas_input_df, sort_orders_df)
    formula_sheet_list = (
        formulas.get_formulas_on_formula_sheet(formulas_input_df))
    
    utilities.delete_directory_if_it_exists(docs_dir)
    formulas.create_formulas_content(formulas_df, formula_sheet_list, 
                            sort_orders_df, docs_dir)
    formulas.create_calculus_summary(formulas_df, formula_sheet_list, 
                            sort_orders_df, docs_dir)
    formulas.create_financial_summary(formulas_df, formula_sheet_list, 
                            sort_orders_df, docs_dir)


In [19]:
import os
import formulas
import utilities
import pandas as pd
import topics

def get_topic_display_string(input_series, **kwarg):
    
    """ !!!!MAY NEED TO UPDATE - not sure if I need to pass topics_df!!!
    Returns a topic summary in markdown format
    Input series is a pandas series with Indices State, Subect code
    and topic.  **Kwarg must be called with parameter  = topics_df
    where topics_df contains fields State,Topic sub category 1, 
    Topic sub category 2, Subject code and Topic
    """
    
    # df = kwarg['topics_df'].copy()
    # df = df[(
    #     (df['State'] == str(input_series['State'])) &
    #     (df['Topic sub category 1'] == str(
    #         input_series['Topic sub category 2'])) &        
    #     (df['Topic sub category 2'] == str(
    #         input_series['Topic sub category 2'])) &
    #     (df['Subject code'] == str(input_series['Subject code'])) & 
    #     (df['Topic'] == str(input_series['Topic'])))]


    output_string=input_series['Topic']
                                          
    return(output_string)


def create_topics_content(topics_df, sort_orders_df, docs_dir):
    """Creates markdown files containing topic tables as input for static 
    web page creation via Hugo
    """

    dirs_df = (
        topics_df[['State', 'Topic sub category 1', 
                             'Topic sub category 2',
                             'Subject code']].drop_duplicates())
    file_paths_df = (
        topics_df[['State', 'Topic sub category 1', 
                             'Topic sub category 2',
                             'Subject code', 'Topic']].drop_duplicates())
    
    utilities.create_sub_directories_from_df(base_dir = docs_dir, 
                                             sub_paths_df = dirs_df)
    
    front_matter_index_files = {'bookCollapseSection' : True}
    utilities.create_index_files(base_dir=docs_dir, dirs_df=dirs_df, 
                       front_matter=front_matter_index_files,
                       sort_orders_df = sort_orders_df)
    
    utilities.create_files(base_dir = docs_dir, file_paths_df= file_paths_df, 
                           file_extension='.md', 
                           fn=get_topic_display_string, 
                           sort_orders_df = sort_orders_df,
                           topics_df = topics_df)




if __name__=='__main__':
    website_creator_dir = os.getcwd()
    docs_dir = utilities.get_docs_path(website_creator_dir)
    order_file_path = website_creator_dir + os.path.sep + 'sort_orders.csv'
    
    sort_orders_df = pd.read_csv(order_file_path)
    topics_df = topics.get_topics_df(sort_orders_df)
    create_topics_content(topics_df, sort_orders_df, docs_dir)
